In [1]:
import pandas as pd

import pickle
from pathlib import Path

In [2]:
month_dir = Path("/projects/frame_align/data/raw/2023-11-01_2023-11-30/")
img_dir = month_dir / "img_data/"

In [3]:
file_sizes = {}
for img_file in img_dir.glob("*.jpg"):
    file_sizes[img_file.name.split(".")[0]] = img_file.stat().st_size
file_size_df = pd.DataFrame.from_dict(file_sizes, orient='index', columns=['file_size'])

In [4]:
file_size_df = file_size_df[file_size_df['file_size'] <= file_size_df['file_size'].quantile(0.95)]
file_size_df = file_size_df[file_size_df['file_size'] >= 5000]
# file_size_df.sort_values('file_size', ascending=True).hist(bins=5)

In [5]:
img_uuids = list(file_size_df.index)

In [6]:
len(img_uuids)

36601

In [7]:
text_df = pd.read_csv(month_dir/"datawithtopiclabels.csv")

In [8]:
text_df.columns

Index(['topic', 'auto_topic_label', 'id', 'authors', 'date_download',
       'date_modify', 'date_publish', 'description', 'filename', 'image_url',
       'language', 'localpath', 'maintext', 'source_domain', 'title',
       'title_page', 'title_rss', 'url', 'month', 'topic_label'],
      dtype='object')

In [9]:
text_df.dropna(subset=['maintext'], inplace=True)
text_df['text_length'] = text_df['maintext'].apply(lambda x: len(x))

In [10]:
lower_quantile = text_df['text_length'].quantile(0.05)
upper_quantile = text_df['text_length'].quantile(0.95)

text_df = text_df[text_df['text_length'] >= lower_quantile]
text_df = text_df[text_df['text_length'] <= upper_quantile]

In [17]:
filtered_text_df = []
for topic_label, topic_df in text_df.groupby('topic_label'):
    if len(topic_df) > 5000:
        topic_df = topic_df.sample(5000)
    filtered_text_df.append(topic_df)
filtered_text_df = pd.concat(filtered_text_df)

In [19]:
len(filtered_text_df)

17084

In [20]:
text_uuids = list(filtered_text_df.id)

In [21]:
common_uuids = set(text_uuids).intersection(set(img_uuids))

In [22]:
len(common_uuids)

16140

In [23]:
pickle.dump(common_uuids, open(month_dir/"selected_uuids.pkl", "wb"))